In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
import os
from sklearn.metrics import confusion_matrix
import json


def read_tfrecord(example):
    input_dim = 29 * 29
    feature_description = {
    'input_features': tf.io.FixedLenFeature([input_dim], tf.int64),
    'label': tf.io.FixedLenFeature([1], tf.int64)
    }
    return tf.io.parse_single_example(example, feature_description)

2024-09-29 17:01:56.700886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 17:01:56.809600: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 17:01:56.839645: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 17:01:57.334322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [2]:
@tf.function
def data_from_tfrecord(tf_filepath, batch_size, repeat_time, shuffle=True):
    data = tf.data.Dataset.from_tensor_slices(tf_filepath)
    data = data.interleave(lambda x: tf.data.TFRecordDataset(x),cycle_length=len(tf_filepath), block_length=10000)
    if shuffle:
        data = data.shuffle(100000, reshuffle_each_iteration=True)
    data = data.map(read_tfrecord, num_parallel_calls=64)
    data = data.repeat(repeat_time + 1)
    data = data.batch(batch_size)
    data = data.prefetch(1)
    # iterator = data.make_one_shot_iterator()
    tf.compat.v1.disable_eager_execution()
    iterator = data.__iter__()
    return iterator.get_next()

In [19]:
def construct_data_flow():
    labels = ['DoS', 'Fuzzy', 'gear', 'RPM']
    data_dir='../Data/'
    batch_size = 2621
    batch_size_unknown = 0
    n_epochs = 1
    unknown_attack = None
    train_unlabel_paths = ['{}/Train_0.7_Labeled_0.1/{}/train_unlabel'.format(data_dir, l) for l in labels]
    #unknown_train_unlabel_path = ['./Data/{}/train_unlabel'.format(unknown_attack)]
    train_label_paths = ['{}/Train_0.7_Labeled_0.1/{}/train_label'.format(data_dir, l) for l in labels if l != unknown_attack]
    val_paths = ['{}/Train_0.7_Labeled_0.1/{}/val'.format(data_dir, l) for l in labels if l != unknown_attack]
    
    print('Unlabeled data: ', train_unlabel_paths)
    print('Label data:', train_label_paths)
    
    train_unlabel = data_from_tfrecord(train_unlabel_paths, batch_size - batch_size_unknown, n_epochs)
    # train_unlabel_unknown = data_from_tfrecord(unknown_train_unlabel_path, batch_size_unknown, n_epochs)
    train_label = data_from_tfrecord(train_label_paths, batch_size, n_epochs)
    validation = data_from_tfrecord(val_paths, batch_size, n_epochs)
    
    
    return train_unlabel, train_label, validation


In [20]:
train_unlabel, train_label, validation = construct_data_flow()

Unlabeled data:  ['../Data//Train_0.7_Labeled_0.1/DoS/train_unlabel', '../Data//Train_0.7_Labeled_0.1/Fuzzy/train_unlabel', '../Data//Train_0.7_Labeled_0.1/gear/train_unlabel', '../Data//Train_0.7_Labeled_0.1/RPM/train_unlabel']
Label data: ['../Data//Train_0.7_Labeled_0.1/DoS/train_label', '../Data//Train_0.7_Labeled_0.1/Fuzzy/train_label', '../Data//Train_0.7_Labeled_0.1/gear/train_label', '../Data//Train_0.7_Labeled_0.1/RPM/train_label']


In [21]:
np.unique(np.array(train_label['label']), return_counts = True)

(array([1]), array([2621]))

In [27]:
np.unique(np.array(validation['label']), return_counts=True)

(array([1]), array([2621]))